In [1]:
# This notebook is a trial to mask raw Cordex data and extract timeseries

In [2]:
# navigate to correct location to access modules

In [3]:
cd /g/data/mn51/users/gt3409/plotting_maps/

/g/data/mn51/users/gt3409/plotting_maps


In [4]:
# import needed packages
from acs_area_statistics import acs_regional_stats, regions
import xarray as xr

In [5]:
ds = xr.open_dataset("/g/data/py18/BARPA/output/CMIP6/DD/AUS-15/BOM/CESM2/historical/r11i1p1f1/BARPA-R/v1-r1/day/pr/v20231001/pr_AUS-15_CESM2_historical_r11i1p1f1_BOM_BARPA-R_v1-r1_day_196001-196012.nc")
ds

<xarray.Dataset> Size: 989MB
Dimensions:    (time: 365, lat: 436, lon: 777, bnds: 2)
Coordinates:
  * time       (time) object 3kB 1960-01-01 12:00:00 ... 1960-12-31 12:00:00
  * lat        (lat) float64 3kB -53.58 -53.42 -53.27 ... 13.32 13.48 13.63
  * lon        (lon) float64 6kB 88.04 88.19 88.34 88.5 ... 207.6 207.8 207.9
Dimensions without coordinates: bnds
Data variables:
    pr         (time, lat, lon) float64 989MB ...
    time_bnds  (time, bnds) object 6kB ...
Attributes: (12/57)
    axiom_version:             0.1.0
    axiom_schemas_version:     0.1.0
    axiom_schema:              cordex-1D.json
    productive_version:        ffe7c4c
    Conventions:               CF-1.10, ACDD-1.3
    activity_id:               RCM
    ...                        ...
    history:                   Thu Nov 09 12:59:42 2023: /g/data/access/ngm/m...
    references:                https://doi.org/10.25914/z1x6-dq28
    license:                   https://doi.org/10.25914/z1x6-dq28
    acknowledgement:           The production of BARPA was supported with fun...
    date_modified:             2023-11-16T20:42:50Z
    date_metadata_modified:    2023-11-16T20:42:50Z

In [6]:
# Adjust lat and lon to correct for float problems! 
ds = ds.assign(lat = ds.lat.round(4), lon = ds.lon.round(4))
ds

<xarray.Dataset> Size: 989MB
Dimensions:    (time: 365, lat: 436, lon: 777, bnds: 2)
Coordinates:
  * time       (time) object 3kB 1960-01-01 12:00:00 ... 1960-12-31 12:00:00
  * lat        (lat) float64 3kB -53.58 -53.42 -53.27 ... 13.32 13.48 13.63
  * lon        (lon) float64 6kB 88.04 88.19 88.34 88.5 ... 207.6 207.8 207.9
Dimensions without coordinates: bnds
Data variables:
    pr         (time, lat, lon) float64 989MB ...
    time_bnds  (time, bnds) object 6kB ...
Attributes: (12/57)
    axiom_version:             0.1.0
    axiom_schemas_version:     0.1.0
    axiom_schema:              cordex-1D.json
    productive_version:        ffe7c4c
    Conventions:               CF-1.10, ACDD-1.3
    activity_id:               RCM
    ...                        ...
    history:                   Thu Nov 09 12:59:42 2023: /g/data/access/ngm/m...
    references:                https://doi.org/10.25914/z1x6-dq28
    license:                   https://doi.org/10.25914/z1x6-dq28
    acknowledgement:           The production of BARPA was supported with fun...
    date_modified:             2023-11-16T20:42:50Z
    date_metadata_modified:    2023-11-16T20:42:50Z

In [7]:
ds = ds.groupby(ds.time.dt.month).sum()

INFO:flox:Entering _validate_reindex: reindex is None
INFO:flox:Leaving _validate_reindex: method = None, returning None
INFO:flox:_choose_engine: Choosing 'flox'


In [8]:
# calculate the masks

In [9]:
%%time
mask = regions.mask_3D(ds)

CPU times: user 6.66 s, sys: 426 ms, total: 7.08 s
Wall time: 7.1 s


/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/regionmask/core/mask.py:444: UserWarning: Detected overlapping regions. As of v0.11.0 these are correctly taken into account. Note, however, that a different mask is returned than with older versions of regionmask. To suppress this warning, set `overlap=True` (to restore the old, incorrect, behaviour, set `overlap=False`).
  warnings.warn(


In [10]:
%%time
# Fractional masking does not work unless the lat lons are a regular grid. 
frac_mask = regions.mask_3D_frac_approx(ds)

CPU times: user 18.2 s, sys: 1.28 s, total: 19.5 s
Wall time: 19.5 s


In [11]:
var = "pr"

In [12]:
%%time
# Apply the mask and calculate regional means for all timesteps
da_means = acs_regional_stats(ds, var=var, dims=("lat", "lon"), mask = frac_mask, how = ["mean"], )
da_means

CPU times: user 504 ms, sys: 128 ms, total: 631 ms
Wall time: 701 ms


<xarray.Dataset> Size: 3kB
Dimensions:  (month: 12, region: 10)
Coordinates:
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * region   (region) int64 80B 0 1 2 3 4 5 6 7 8 9
    abbrevs  (region) <U8 320B 'WANorth' 'WASouth' 'NSW' ... 'QLDSouth' 'AUS'
    names    (region) <U48 2kB 'Western Australia North' ... 'Australia'
Data variables:
    pr_mean  (month, region) float64 960B 0.0004511 9.92e-05 ... 0.0006641

In [13]:
# Plot the timeseries of one of the regions
da_means.sel(region = 0)

<xarray.Dataset> Size: 424B
Dimensions:  (month: 12)
Coordinates:
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
    region   int64 8B 0
    abbrevs  <U8 32B 'WANorth'
    names    <U48 192B 'Western Australia North'
Data variables:
    pr_mean  (month) float64 96B 0.0004511 0.0003408 ... 0.0002543 0.0004091